# Bandicoot notebook

_bandicoot_ is an open-source python toolbox to analyze mobile phone metadata. For more information, see: http://bandicoot.mit.edu/

The source code of the notebook is available as ``demo.ipynb`` and a plain
Python version as ``demo.py``. You can download them from our repository on Github at [https://github.com/yvesalexandre/bandicoot/tree/master/demo](https://github.com/yvesalexandre/bandicoot/tree/master/demo)


<br />
<div class="alert alert-info" role="alert">
<p><strong>Try bandicoot on your phone!</strong></p>
<p>If you want to try bandicoot with your own data, download our Android app at <a href="http://bandicoot.mit.edu/android">bandicoot.mit.edu/android</a></p>
</div>

<hr>

### Input files

<img src="mini-mockups-01.png" width="80%" style="border: 1px solid #aaa" />


In [1]:
# Records for the user 'ego'
!head -n 5 data/ego.csv

interaction,direction,correspondent_id,datetime,call_duration,antenna_id
text,in,A,2014-03-02 07:13:30,,1
text,in,E,2014-03-02 07:53:30,,1
text,in,E,2014-03-02 08:22:30,,2
text,out,D,2014-03-02 08:34:30,,3


In [2]:
# GPS locations of cell towers
!head -n 5 data/antennas.csv

antenna_id,latitude,longitude
1,42.366944,-71.083611
2,42.386722,-71.138778
3,42.3604,-71.087374
4,42.353917,-71.105


# Loading a user

In [3]:
import bandicoot as bc

U = bc.read_csv('ego', 'data/', 'data/antennas.csv')

[x] 314 records from 2014-03-02 07:13:30 to 2014-04-14 12:04:37
[x] 7 contacts
[ ] No attribute stored
[x] 27 antennas
[ ] No recharges
[x] Has home
[x] Has texts
[x] Has calls
[ ] No network


# Visualization

Export and serve an interactive visualization using:

```python
bc.visualization.run(U)
```

or export only using:

```python
bc.visualization.export(U, 'my-viz-path')
```

In [4]:
import os
viz_path = os.path.dirname(os.path.realpath(__name__)) + '/viz'

bc.visualization.export(U, viz_path)

Successfully exported the visualization to /Volumes/Data/projects/bandicoot/demo/viz


'/Volumes/Data/projects/bandicoot/demo/viz'

In [5]:
from IPython.display import IFrame
IFrame("/files/viz/index.html", "100%", 700)

## Individual and spatial indicators

Using bandicoot, compute aggregated indicators from ``bc.individual`` and ``bc.spatial``: 

In [6]:
bc.individual.percent_initiated_conversations(U)

{
    "allweek": {
        "allday": {
            "callandtext": {
                "mean": 0.3244815064488733, 
                "std": 0.09659866096759165
            }
        }
    }
}

In [7]:
bc.spatial.number_of_antennas(U)

{
    "allweek": {
        "allday": {
            "mean": 5.375, 
            "std": 1.8666480653835098
        }
    }
}

In [8]:
bc.spatial.radius_of_gyration(U)

{
    "allweek": {
        "allday": {
            "mean": 1.4503807789208683, 
            "std": 0.8575480642906887
        }
    }
}

# Let's play with indicators

The signature of the ``active_days`` indicators is:

```python
bc.individual.active_days(user, groupby='week', interaction='callandtext', summary='default', split_week=False, split_day=False, filter_empty=True, datatype=None)
```

What does that mean?


<hr />


## The *‘groupby’* keyword

<br />
<div class="alert alert-info" role="alert">
<p><strong>Weekly aggregation</strong></p>
<p>By default, _bandicoot_ computes the indicators on a weekly basis and returns the average (mean) over all the weeks available and its standard deviation (std) in a nested dictionary.</p>

<img src="mini-mockups-02.png" width="80%" style="border: 1px solid #aaa" />


</div>

In [9]:
bc.individual.active_days(U)

{
    "allweek": {
        "allday": {
            "callandtext": {
                "mean": 5.5, 
                "std": 2.598076211353316
            }
        }
    }
}

The ``groupby`` keyword controls the aggregation:

- ``groupby='week'`` to divide by week (by default),
- ``groupby='month'`` to divide by month,
- ``groupby=None`` to aggregate all values.

In [10]:
bc.individual.active_days(U, groupby='week')

{
    "allweek": {
        "allday": {
            "callandtext": {
                "mean": 5.5, 
                "std": 2.598076211353316
            }
        }
    }
}

In [11]:
bc.individual.active_days(U, groupby='month')

{
    "allweek": {
        "allday": {
            "callandtext": {
                "mean": 22.0, 
                "std": 8.0
            }
        }
    }
}

In [12]:
bc.individual.active_days(U, groupby=None)

{
    "allweek": {
        "allday": {
            "callandtext": 44
        }
    }
}

## The *‘summary’* keyword

Some indicators such as *active\_days* returns one number. Others, such as *duration\_of\_calls* returns a distribution.

The summary keyword can take three values:

 - `summary='default'` to return mean and standard deviation,
 - `summary='extended'` for the second type of indicators, to return mean, sem, median, skewness and std of the distribution,
 - `summary=None` to return the full distribution.

In [13]:
bc.individual.call_duration(U)

{
    "allweek": {
        "allday": {
            "call": {
                "mean": {
                    "mean": 3776.7093501036775, 
                    "std": 1404.827412706482
                }, 
                "std": {
                    "mean": 1633.3931770157765, 
                    "std": 689.2035500056488
                }
            }
        }
    }
}

In [14]:
bc.individual.call_duration(U, summary='extended')

{
    "allweek": {
        "allday": {
            "call": {
                "mean": {
                    "mean": 3776.7093501036775, 
                    "std": 1404.827412706482
                }, 
                "std": {
                    "mean": 1633.3931770157765, 
                    "std": 689.2035500056488
                }, 
                "median": {
                    "mean": 3714.714285714286, 
                    "std": 1532.9148671064283
                }, 
                "skewness": {
                    "mean": 0.12925073170191398, 
                    "std": 0.48628300355189896
                }, 
                "kurtosis": {
                    "mean": 1.8063876957023484, 
                    "std": 0.8998073161683097
                }, 
                "min": {
                    "mean": 1330.857142857143, 
                    "std": 2200.2680634459994
                }, 
                "max": {
                    "mean": 6468.857142857143, 
              

In [15]:
bc.individual.call_duration(U, summary=None)

{
    "allweek": {
        "allday": {
            "call": [
                [
                    374, 
                    1086, 
                    1099, 
                    1330, 
                    2456, 
                    3404, 
                    4472, 
                    5359, 
                    5413, 
                    6233
                ], 
                [
                    594, 
                    1927, 
                    2072, 
                    2258, 
                    2854, 
                    3286, 
                    3552, 
                    4202, 
                    4689, 
                    5142, 
                    5689, 
                    5752, 
                    6429, 
                    6891, 
                    7082, 
                    7123
                ], 
                [
                    403, 
                    539, 
                    2109, 
                    2726, 
                    2871, 
                

## Splitting days and weeks

- **``split_week``** divide records by 'all week', 'weekday', and 'weekend'.
- **``split_day``** divide records by 'all day', 'day', and 'night'.


In [16]:
bc.individual.active_days(U, split_week=True, split_day=True)

{
    "allweek": {
        "allday": {
            "callandtext": {
                "mean": 5.5, 
                "std": 2.598076211353316
            }
        }, 
        "day": {
            "callandtext": {
                "mean": 5.5, 
                "std": 2.598076211353316
            }
        }, 
        "night": {
            "callandtext": {
                "mean": 5.375, 
                "std": 2.54644359843292
            }
        }
    }, 
    "weekday": {
        "allday": {
            "callandtext": {
                "mean": 4.428571428571429, 
                "std": 1.3997084244475304
            }
        }, 
        "day": {
            "callandtext": {
                "mean": 4.428571428571429, 
                "std": 1.3997084244475304
            }
        }, 
        "night": {
            "callandtext": {
                "mean": 4.428571428571429, 
                "std": 1.3997084244475304
            }
        }
    }, 
    "weekend": {
        "allday": {
 

# Exporting indicators

The function ``bc.utils.all`` computes automatically all indicators for a single user.

You can use the same keywords to group by week/month/all time range, or return extended statistics.

In [17]:
features = bc.utils.all(U, groupby=None)

In [18]:
features

{
    "name": "ego", 
    "reporting": {
        "antennas_path": "data/antennas.csv", 
        "attributes_path": None, 
        "recharges_path": None, 
        "version": "0.4.0", 
        "code_signature": "9f648eb0aa73d5e521c7e8a2bf8256697a467bdf", 
        "groupby": None, 
        "split_week": false, 
        "split_day": false, 
        "start_time": "2014-03-02 07:13:30", 
        "end_time": "2014-04-14 12:04:37", 
        "night_start": "19:00:00", 
        "night_end": "07:00:00", 
        "weekend": [
            6, 
            7
        ], 
        "number_of_records": 314, 
        "number_of_antennas": 27, 
        "number_of_recharges": 0, 
        "bins": 1, 
        "bins_with_data": 1, 
        "bins_without_data": 0, 
        "has_call": true, 
        "has_text": true, 
        "has_home": true, 
        "has_recharges": false, 
        "has_attributes": false, 
        "has_network": false, 
        "percent_records_missing_location": 0.0, 
        "antennas_mi

#### Exporting in CSV and JSON

bandicoot supports exports in CSV and JSON format. Both ``to_csv`` and ``to_json`` functions require either a single feature dictionnary, or a list of dictionnaries (for multiple users).

In [19]:
bc.to_csv(features, 'demo_export_user.csv')
bc.to_json(features, 'demo_export_user.json')

Successfully exported 1 object(s) to demo_export_user.csv
Successfully exported 1 object(s) to demo_export_user.json


In [20]:
!head demo_export_user.csv

In [21]:
!head -n 15 demo_export_user.json

{
    "ego": {
        "name": "ego",
        "reporting": {
            "antennas_path": "data/antennas.csv",
            "attributes_path": null,
            "recharges_path": null,
            "version": "0.4.0",
            "code_signature": "9f648eb0aa73d5e521c7e8a2bf8256697a467bdf",
            "groupby": null,
            "split_week": false,
            "split_day": false,
            "start_time": "2014-03-02 07:13:30",
            "end_time": "2014-04-14 12:04:37",
            "night_start": "19:00:00",


# Extending bandicoot

You can easily develop your indicator using the ``@grouping`` decorator. You only need to write a function taking as input a list of records and returning an integer or a list of integers (for a distribution). The ``@grouping`` decorator wraps the function and call it for each group of weeks.

In [22]:
from bandicoot.helper.group import grouping

@grouping(interaction='call')
def shortest_call(records):
    in_durations = (r.call_duration for r in records)
    return min(in_durations)

In [23]:
shortest_call(U)

{
    "allweek": {
        "allday": {
            "call": {
                "mean": 1330.857142857143, 
                "std": 2200.2680634459994
            }
        }
    }
}

In [24]:
shortest_call(U, split_day=True)

{
    "allweek": {
        "allday": {
            "call": {
                "mean": 1330.857142857143, 
                "std": 2200.2680634459994
            }
        }, 
        "day": {
            "call": {
                "mean": 1395.5714285714287, 
                "std": 2186.820187078088
            }
        }, 
        "night": {
            "call": {
                "mean": 1182.1666666666667, 
                "std": 908.7290422464895
            }
        }
    }
}